In [ ]:
from openpyxl import load_workbook

def parse_xlsx(filename, sheetname):
    wb = load_workbook(filename=filename, read_only=True)
    ws = wb[sheetname]

    sheet_rows = list()
    for num, row in enumerate(ws.rows):
        if num == 0:
            headers = [i.value for i in row]
        else:
            row_cells = [i.value for i in row]
            row_dict = dict(zip(headers, row_cells))
            sheet_rows.append(row_dict)
    return sheet_rows

In [ ]:
source_rows = parse_xlsx('Feb13JacobSource/Turnaround Time - Article - to de-dupe.xlsx', 'Sheet')
len(source_rows)

In [ ]:
fulfilleds = [row for row in source_rows if row.get('Changed To') == 'Delivered to Web']
len(fulfilleds)

In [ ]:
fulfilled_trans_nums = {row.get('Transaction Number'): 0 for row in fulfilleds}
len(fulfilled_trans_nums)

In [ ]:
for item in fulfilleds:
    fulfilled_trans_nums[item.get('Transaction Number')] += 1

In [ ]:
duplicate_fulfilleds = {k for k, v in fulfilled_trans_nums.items() if v > 1}
len(duplicate_fulfilleds)

In [ ]:
no_delivered = list()
just_done = set()


for item in source_rows:
    trans_num = item.get('Transaction Number')
    if not trans_num:
        print('whoa', item)
    else:
        if trans_num not in fulfilled_trans_nums:
            if trans_num in just_done:
                continue
            else:
                no_delivered.append(item)
                just_done.add(trans_num)

In [ ]:
A = [i for i in source_rows if i.get('Changed To') != 'Request Finished']
len(A)

In [ ]:
B = [i for i in A if i.get('Transaction Number') not in duplicate_fulfilleds]
len(B)

In [ ]:
C = B + no_delivered
len(C)

In [ ]:
holder = dict()
for i in C:
    trans_num = i.get('Transaction Number')
    if trans_num in holder:
        holder[trans_num] += 1
    else:
        holder[trans_num] = 1

In [ ]:
for k,v in holder.items():
    if v != 1:
        print(k)

In [ ]:
import csv

with open('/home/raleigh/Desktop/lsugit/ScrapingWebpages/MatchingIlsEbooks/Feb13JacobSource_output.csv', 'w', newline='') as csvfile:
    fieldnames = C[0].keys()
    spamwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    spamwriter.writeheader()
    for row in C:
        spamwriter.writerow(row)